In [ ]:
# @title 連接google drive
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi
import os
os.chdir('/content/drive/My Drive')
os.getcwd()
os.chdir('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data')
os.getcwd()

Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found


'/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
!pip install networkx polyline
import networkx as nx
import polyline
import ast

In [ ]:

df = pd.read_csv("file_fi2.csv")
print(df)
print(df.info())
# df.describe(include="all")
print("object行:\n",df.select_dtypes(include="object").columns)
print("缺失值:\n",df.isnull().sum())
print("唯一值:\n",df[:].nunique())

            ID Delivery_person_ID  Delivery_person_Age  \
0      0x4607     INDORES13DEL02                  37.0   
1      0xb379     BANGRES18DEL02                  34.0   
2      0x5d6d     BANGRES19DEL01                  23.0   
3      0x7a6a    COIMBRES13DEL02                  38.0   
4      0x70a2     CHENRES12DEL01                  32.0   
...        ...                ...                  ...   
40304  0x1178   RANCHIRES16DEL01                  35.0   
40305  0x7c09      JAPRES04DEL01                  30.0   
40306  0x4f8d     CHENRES08DEL03                  30.0   
40307  0x5eee    COIMBRES11DEL01                  20.0   
40308  0x5fb2   RANCHIRES09DEL02                  23.0   

       Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  \
0                          4.9            22.745049             75.892471   
1                          4.5            12.913041             77.683237   
2                          4.4            12.914264             77.67840

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import numpy as np



# 转换时间数据
def convert_time(df, columns):
    for col in tqdm(columns, desc="Converting Time"):
        # 创建新列以存储转换后的时间数据
        df[col + '_in_minutes'] = pd.to_datetime(df[col], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(df[col], format='%H:%M:%S').dt.minute
    return df

# 聚类地理位置
def cluster_locations(df, lat_cols, lon_cols, n_clusters=22):
    # 合并纬度和经度列
    locations = df[lat_cols + lon_cols].values
    # 标准化
    scaler = StandardScaler()
    locations_scaled = scaler.fit_transform(locations)
    # 聚类
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(locations_scaled)
    return kmeans.labels_

# 聚类时间
def cluster_times(df, time_col, n_clusters=3):
    # 将一维数组转换为二维数组
    times = df[time_col].values.reshape(-1, 1)
    # 标准化
    scaler = StandardScaler()
    times_scaled = scaler.fit_transform(times)
    # 聚类
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(times_scaled)
    return kmeans.labels_

# 转换时间列
time_cols = ['Time_Orderd']
df = convert_time(df, time_cols)

# 地理位置聚类
location_cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']
df['location_cluster'] = cluster_locations(df, location_cols[:2], location_cols[2:])

# # 时间聚类
# time_cluster_col = 'Time_Orderd_in_minutes'  # 新的时间列
# df['time_cluster'] = cluster_times(df, time_cluster_col)

# 查看结果
# print(df[['location_cluster', 'time_cluster']])


Converting Time: 100%|██████████| 1/1 [00:00<00:00, 42.80it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import pandas as pd

# 假设 df 是您的 DataFrame

# 确保 Order_Date 是日期格式
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')

# 为每个日期分配一个唯一的类别标签
df['date_category'] = df.groupby('Order_Date').ngroup()

# 查看结果
print(df[['Order_Date', 'date_category']])


      Order_Date  date_category
0     2022-03-19             26
1     2022-03-25             31
2     2022-03-19             26
3     2022-04-05             42
4     2022-03-26             32
...          ...            ...
40304 2022-03-08             15
40305 2022-03-24             30
40306 2022-03-11             18
40307 2022-03-07             14
40308 2022-03-02              9

[40309 rows x 2 columns]


In [ ]:
import folium
from tqdm import tqdm

# Specific date category selection
specific_date_category = 1

# Filter data for the specific day
specific_day_df = df[df['date_category'] == specific_date_category]

# Create a Folium map instance
m = folium.Map(location=[specific_day_df['Restaurant_latitude'].mean(), specific_day_df['Restaurant_longitude'].mean()], zoom_start=12)

# Define two sets of colors for restaurants and delivery points
restaurant_colors = ['red', 'blue', 'green', 'purple', 'orange']
delivery_colors = ['darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']

# Add markers for each location with progress bar
for _, row in tqdm(specific_day_df.iterrows(), total=specific_day_df.shape[0], desc="Adding Markers"):
    location_cluster = row['location_cluster']

    # Restaurant location
    folium.CircleMarker(
        location=[row['Restaurant_latitude'], row['Restaurant_longitude']],
        radius=5,
        color=restaurant_colors[location_cluster % len(restaurant_colors)],
        fill=True,
        fill_color=restaurant_colors[location_cluster % len(restaurant_colors)],
        popup=f"Restaurant - Cluster: {location_cluster},  Ordered: {row['Time_Orderd']}"
    ).add_to(m)

    # Delivery location
    folium.CircleMarker(
        location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
        radius=5,
        color=delivery_colors[location_cluster % len(delivery_colors)],
        fill=True,
        fill_color=delivery_colors[location_cluster % len(delivery_colors)],
        popup=f"Delivery Point - Cluster: {location_cluster}, Ordered: {row['Time_Orderd']}"
    ).add_to(m)

# Display the map
m


Adding Markers: 100%|██████████| 683/683 [00:00<00:00, 2255.60it/s]


In [ ]:
def calculate_distance(point1, point2):
    # 计算两点之间的距离
    lat_diff = abs(point1[0] - point2[0])
    lon_diff = abs(point1[1] - point2[1])
    return lat_diff + lon_diff

def update_shortest_distances(route):
    # 初始化最短距离列表，初始值设为无限大
    shortest_distances = [float('inf')] * len(route)

    # 对于路线中的每个点，计算到其他所有点的距离
    for i, point1 in enumerate(route):
        for j, point2 in enumerate(route):
            print(i,j)

    return shortest_distances

# 应用到DataFrame中
# df.at[0,'Route'].apply(update_shortest_distances)
update_shortest_distances(df.at[0,'Route'])
# 显示结果
df.head()

In [ ]:
df.to_csv('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/fi5.csv', index=False)

In [ ]:

df = pd.read_csv("fi5.csv")
# df.at[0,'Route']
# df.at[0, 'Route'][0]
# 将字符串转换为列表
route_list = ast.literal_eval(df.at[0, 'Route'])
def calculate_shortest_path(start_point, end_point):
    # 简化的距离计算，使用经纬度差的绝对值
    lat_diff = abs(start_point[0] - end_point[0])
    lon_diff = abs(start_point[1] - end_point[1])
    return lat_diff + lon_diff
# 现在 route_list 是一个包含经纬度坐标的列表
# print(route_list)
route_list[1]
# for coordinate in route_list:
#     print(coordinate[0])
    # for value in coordinate:
    #     print(value)  # 这将会打印出每个坐标的纬度和经度值
# print(df['date_category'].unique())

date_categories = df['date_category'].unique()
location_clusters = df['location_cluster'].unique()
# 指定您的目录路径
directory_path = '/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/'

for date_category in tqdm(date_categories, desc='Processing date categories'):
    for location_cluster in tqdm(location_clusters, desc=f'Processing location clusters for date category {date_category}'):
        specific_df = df[(df['date_category'] == date_category) &
                         (df['location_cluster'] == location_cluster)]
        file_name = f'{date_category}_{location_cluster}_df.csv'
        file_path = os.path.join(directory_path, file_name)

        # 检查文件是否存在
        if os.path.exists(file_path):
            print(f"文件 {file_name} 已存在。")
            continue
        else:
            print(f"文件 {file_name} 不存在，需要执行操作。")

        # 用于存储每行的短路径信息的列表
        rows_to_add = []
        shortest_paths_df = pd.DataFrame()
        for index, row in tqdm(specific_df.iterrows(), total=specific_df.shape[0], desc='Calculating shortest paths'):
            route_list = ast.literal_eval(row['Route'])
            shortest_paths_row = {'original_index': index}  # 将原始索引添加到字典中


            for coordinate in route_list:
                for index2, row2 in specific_df.iterrows():
                    if index == index2:
                        continue
                    restaurant_location2 = (row2['Restaurant_latitude'], row2['Restaurant_longitude'])
                    delivery_location2 = (row2['Delivery_location_latitude'], row2['Delivery_location_longitude'])
                    shortest_path_length_restaurant = calculate_shortest_path(coordinate, restaurant_location2)
                    shortest_path_length_delivery = calculate_shortest_path(coordinate, delivery_location2)

                    # 检查并更新最短路径长度
                    current_shortest_restaurant = shortest_paths_row.get((index2, 'restaurant'), float('inf'))
                    current_shortest_delivery = shortest_paths_row.get((index2, 'delivery'), float('inf'))

                    if shortest_path_length_restaurant < current_shortest_restaurant:
                        shortest_paths_row[(index2, 'restaurant')] = shortest_path_length_restaurant
                    if shortest_path_length_delivery < current_shortest_delivery:
                        shortest_paths_row[(index2, 'delivery')] = shortest_path_length_delivery

            # 将字典转换为数据框，并存储到列表中
            row_df = pd.DataFrame([shortest_paths_row])
            rows_to_add.append(row_df)

        # 使用pandas.concat来添加所有行到shortest_paths_df
        shortest_paths_df = pd.concat([shortest_paths_df] + rows_to_add, ignore_index=True)

        print("Completed for date category", date_category, "and location cluster", location_cluster)
        shortest_paths_df.head()
        shortest_paths_df.to_csv(f'/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filter_data/{date_category}_{location_cluster}_df.csv', index=False)


Processing date categories:   0%|          | 0/44 [00:00<?, ?it/s]
Processing location clusters for date category 26:   0%|          | 0/22 [00:00<?, ?it/s]

文件 26_1_df.csv 已存在。
文件 26_2_df.csv 已存在。
文件 26_10_df.csv 已存在。
文件 26_8_df.csv 已存在。
文件 26_9_df.csv 已存在。
文件 26_5_df.csv 已存在。
文件 26_15_df.csv 已存在。
文件 26_4_df.csv 不存在，需要执行操作。




Calculating shortest paths: 0it [00:00, ?it/s]
Processing location clusters for date category 26: 100%|██████████| 22/22 [00:00<00:00, 256.96it/s]
Processing date categories:   2%|▏         | 1/44 [00:00<00:04,  9.89it/s]

Completed for date category 26 and location cluster 4
文件 26_21_df.csv 已存在。
文件 26_12_df.csv 已存在。
文件 26_13_df.csv 已存在。
文件 26_16_df.csv 已存在。
文件 26_3_df.csv 已存在。
文件 26_0_df.csv 已存在。
文件 26_6_df.csv 已存在。
文件 26_11_df.csv 已存在。
文件 26_14_df.csv 已存在。
文件 26_17_df.csv 已存在。
文件 26_7_df.csv 已存在。
文件 26_20_df.csv 已存在。
文件 26_18_df.csv 已存在。
文件 26_19_df.csv 已存在。



Processing location clusters for date category 31: 100%|██████████| 22/22 [00:00<00:00, 426.38it/s]


...


Processing location clusters for date category 43:   0%|          | 0/22 [00:00<?, ?it/s]

文件 43_1_df.csv 已存在。
文件 43_2_df.csv 已存在。
文件 43_10_df.csv 已存在。
文件 43_8_df.csv 已存在。
文件 43_9_df.csv 已存在。
文件 43_5_df.csv 已存在。
文件 43_15_df.csv 已存在。
文件 43_4_df.csv 已存在。
文件 43_21_df.csv 已存在。
文件 43_12_df.csv 已存在。
文件 43_13_df.csv 已存在。
文件 43_16_df.csv 已存在。
文件 43_3_df.csv 已存在。
文件 43_0_df.csv 已存在。
文件 43_6_df.csv 已存在。
文件 43_11_df.csv 已存在。
文件 43_14_df.csv 已存在。
文件 43_17_df.csv 已存在。
文件 43_7_df.csv 已存在。
文件 43_20_df.csv 已存在。
文件 43_18_df.csv 已存在。


Processing location clusters for date category 43: 100%|██████████| 22/22 [00:00<00:00, 224.31it/s]

文件 43_19_df.csv 已存在。



Processing date categories: 100%|██████████| 44/44 [00:03<00:00, 11.93it/s]


In [ ]:
df = pd.read_csv("fi5.csv")
for date_category in tqdm(date_categories, desc='Processing date categories'):
    for location_cluster in tqdm(location_clusters, desc=f'Processing location clusters for date category {date_category}'):
        specific_df = df[(df['date_category'] == date_category) &
                         (df['location_cluster'] == location_cluster)]

        time_diffs_df = pd.DataFrame()
        rows_to_add = []

        for index, row in tqdm(specific_df.iterrows(), total=specific_df.shape[0], desc='Calculating time differences'):
            order_time = row['Time_Orderd_in_minutes']
            time_diffs_row = {'original_index': index}

            for index2, row2 in specific_df.iterrows():
                if index == index2:
                    continue
                other_order_time = row2['Time_Orderd_in_minutes']
                time_diff = abs(order_time - other_order_time)

                time_diffs_row[index2] = time_diff

            # 将字典转换为数据框，并存储到列表中
            row_df = pd.DataFrame([time_diffs_row])
            rows_to_add.append(row_df)

        # 使用pandas.concat来添加所有行到time_diffs_df
        time_diffs_df = pd.concat([time_diffs_df] + rows_to_add, ignore_index=True)

        print("Completed for date category", date_category, "and location cluster", location_cluster)
        time_diffs_df.head()
        time_diffs_df.to_csv(f'/content/drive/My Drive/碳數據科技黑客松/t/t{date_category}_{location_cluster}_time_diffs.csv', index=False)


Processing date categories:   0%|          | 0/44 [00:00<?, ?it/s]
Processing location clusters for date category 26:   0%|          | 0/22 [00:00<?, ?it/s]

Calculating time differences:   0%|          | 0/84 [00:00<?, ?it/s]

Calculating time differences:  17%|█▋        | 14/84 [00:00<00:00, 138.79it/s]

Calculating time differences:  36%|███▌      | 30/84 [00:00<00:00, 147.37it/s]

Calculating time differences:  55%|█████▍    | 46/84 [00:00<00:00, 152.83it/s]

Calculating time differences:  74%|███████▍  | 62/84 [00:00<00:00, 155.12it/s]

Calculating time differences: 100%|██████████| 84/84 [00:00<00:00, 149.59it/s]


Completed for date category 26 and location cluster 1



Processing location clusters for date category 26:   5%|▍         | 1/22 [00:01<00:22,  1.08s/it]

Calculating time differences:   0%|          | 0/102 [00:00<?, ?it/s]

Calculating time differences:  12%|█▏        | 12/102 [00:00<00:00, 115.90it/s]

Calculating time differences:  25%|██▌       | 26/102 [00:00<00:00, 126.20it/s]

Calculating time differences:  38%|███▊      | 39/102 [00:00<00:00, 123.06it/s]

Calculating time differences:  51%|█████     | 52/102 [00:00<00:00, 124.21it/s]

Calculating time differences:  64%|██████▎   | 65/102 [00:00<00:00, 125.58it/s]

Calculating time differences:  76%|███████▋  | 78/102 [00:00<00:00, 124.33it/s]

Calculating time differences: 100%|██████████| 102/102 [00:00<00:00, 125.25it/s]

Processing location clusters for date category 26:   9%|▉         | 2/22 [00:02<00:23,  1.17s/it]

Completed for date category 26 and location cluster 2




Calculating time differences:   0%|          | 0/98 [00:00<?, ?it/s]

Calculating time differences:  15%|█▌        | 15/98 [00:00<00:00, 143.22it/s]

Calculating time differences:  31%|███       | 30/98 [00:00<00:00, 129.70it/s]

Calculating time differences:  45%|████▍     | 44/98 [00:00<00:00, 130.24it/s]

Calculating time differences:  59%|█████▉    | 58/98 [00:00<00:00, 133.43it/s]

Calculating time differences:  73%|███████▎  | 72/98 [00:00<00:00, 134.85it/s]

Calculating time differences: 100%|██████████| 98/98 [00:00<00:00, 134.16it/s]

Processing location clusters for date category 26:  14%|█▎        | 3/22 [00:03<00:21,  1.11s/it]

...

Completed for date category 43 and location cluster 20




Calculating time differences: 0it [00:00, ?it/s]


Completed for date category 43 and location cluster 18




Calculating time differences: 0it [00:00, ?it/s]
Processing location clusters for date category 43: 100%|██████████| 22/22 [00:12<00:00,  1.71it/s]
Processing date categories: 100%|██████████| 44/44 [09:56<00:00, 13.56s/it]

Completed for date category 43 and location cluster 19
